## Set up & import dependencies

In [1]:
# Set up and import dependencies
import pandas as pd
from sqlalchemy import create_engine
import re

import warnings
warnings.filterwarnings("ignore")

## Extract CSV data into DataFrame

In [2]:
# Read input csv datafile
uni_2022 = pd.read_csv("Resources/QS World University Rankings 2018.csv")
uni_2022.head()

,year,rank_display,university,score,link,country,city,region,logo
0,2018,1,Massachusetts Institute of Technology (MIT),100.0,https://www.topuniversities.com/universities/m...,United States,Cambridge,North America,https://www.topuniversities.com/sites/default/...
1,2018,2,Stanford University,98.7,https://www.topuniversities.com/universities/s...,United States,Stanford,North America,https://www.topuniversities.com/sites/default/...
2,2018,3,Harvard University,98.4,https://www.topuniversities.com/universities/h...,United States,Cambridge,North America,https://www.topuniversities.com/sites/default/...
3,2018,4,California Institute of Technology (Caltech),97.7,https://www.topuniversities.com/universities/c...,United States,Pasadena,North America,https://www.topuniversities.com/sites/default/...
4,2018,5,University of Cambridge,95.6,https://www.topuniversities.com/universities/u...,United Kingdom,Cambridge,Europe,https://www.topuniversities.com/sites/default/...


In [3]:
# Create pandas dataframe to hold the csv data
uni_rank_2022 = []
uni_rank_2022 = uni_2022[["year", "rank_display","university","score","country","city","region","link"]].copy()

## Analyse DataFrame

In [4]:
uni_rank_2022.reset_index(drop=True,inplace=True)
uni_rank_2022

,year,rank_display,university,score,country,city,region,link
0,2018,1,Massachusetts Institute of Technology (MIT),100.0,United States,Cambridge,North America,https://www.topuniversities.com/universities/m...
1,2018,2,Stanford University,98.7,United States,Stanford,North America,https://www.topuniversities.com/universities/s...
2,2018,3,Harvard University,98.4,United States,Cambridge,North America,https://www.topuniversities.com/universities/h...
3,2018,4,California Institute of Technology (Caltech),97.7,United States,Pasadena,North America,https://www.topuniversities.com/universities/c...
4,2018,5,University of Cambridge,95.6,United Kingdom,Cambridge,Europe,https://www.topuniversities.com/universities/u...
...,...,...,...,...,...,...,...,...
972,2018,1001+,Université de Technologie de Compiègne (UTC),NaN,France,Compiègne,Europe,https://www.topuniversities.com/universities/u...
973,2018,1001+,"University of California, San Francisco",NaN,United States,San Francisco,North America,https://www.topuniversities.com/universities/u...
974,2018,1001+,Prague University of Economics and Business,NaN,Czech Republic,Prague,Europe,https://www.topuniversities.com/universities/p...
975,2018,1001+,Weizmann Institute of Science,NaN,Israel,Rehovot,Asia,https://www.topuniversities.com/universities/w...


In [5]:
# Check for duplicated data
uni_rank_2022.duplicated().sum()

0

In [6]:
# Check for a NaN in the entire dataframe
uni_rank_2022.isnull().values.any()

True

In [7]:
# Locate and count the NaNs in the dataframe columns
uni_rank_2022.isnull().sum()

year              0
rank_display      1
university        0
score           577
country           0
city             17
region            0
link              0
dtype: int64

In [8]:
# Get information of all columns in the dataframe
uni_rank_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          977 non-null    int64  
 1   rank_display  976 non-null    object 
 2   university    977 non-null    object 
 3   score         400 non-null    float64
 4   country       977 non-null    object 
 5   city          960 non-null    object 
 6   region        977 non-null    object 
 7   link          977 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 61.2+ KB


## Transforming data (cleasing) - Column: City

In [9]:
# Locate the position of the string that may contain city name in the university link
len("https://www.topuniversities.com/universities/")

45

In [10]:
# Extract the string that contains city name from the url and add to the dataframe as a new column
uni_rank_2022["city_extract"] = uni_rank_2022['link'].str[45:]
uni_rank_2022

,year,rank_display,university,score,country,city,region,link,city_extract
0,2018,1,Massachusetts Institute of Technology (MIT),100.0,United States,Cambridge,North America,https://www.topuniversities.com/universities/m...,massachusetts-institute-technology-mit
1,2018,2,Stanford University,98.7,United States,Stanford,North America,https://www.topuniversities.com/universities/s...,stanford-university
2,2018,3,Harvard University,98.4,United States,Cambridge,North America,https://www.topuniversities.com/universities/h...,harvard-university
3,2018,4,California Institute of Technology (Caltech),97.7,United States,Pasadena,North America,https://www.topuniversities.com/universities/c...,california-institute-technology-caltech
4,2018,5,University of Cambridge,95.6,United Kingdom,Cambridge,Europe,https://www.topuniversities.com/universities/u...,university-cambridge
...,...,...,...,...,...,...,...,...,...
972,2018,1001+,Université de Technologie de Compiègne (UTC),NaN,France,Compiègne,Europe,https://www.topuniversities.com/universities/u...,universite-de-technologie-de-compiegne-utc
973,2018,1001+,"University of California, San Francisco",NaN,United States,San Francisco,North America,https://www.topuniversities.com/universities/u...,university-california-san-francisco
974,2018,1001+,Prague University of Economics and Business,NaN,Czech Republic,Prague,Europe,https://www.topuniversities.com/universities/p...,prague-university-economics-business
975,2018,1001+,Weizmann Institute of Science,NaN,Israel,Rehovot,Asia,https://www.topuniversities.com/universities/w...,weizmann-institute-science


In [11]:
# Drop the unwanted link column
uni_rank_2022 = uni_rank_2022.drop(["link"], axis=1)

# Locate the NaN value in the city column
uni_rank_2022[uni_rank_2022["city"].isnull()]

,year,rank_display,university,score,country,city,region,city_extract
118,2018,119,Aarhus University,59.3,Denmark,NaN,Europe,aarhus-university
201,2018,202,Queen's University Belfast,47.4,United Kingdom,NaN,Europe,queens-university-belfast
241,2018,242,Université Paris-Saclay,42.3,France,NaN,Europe,universite-paris-saclay
256,2018,=256,Kyung Hee University,40.6,South Korea,NaN,Asia,kyung-hee-university
346,2018,=346,Brunel University London,33.9,United Kingdom,NaN,Europe,brunel-university-london
360,2018,=361,Oxford Brookes University,33.1,United Kingdom,NaN,Europe,oxford-brookes-university
385,2018,=386,National Research Tomsk Polytechnic University,31.3,Russia,NaN,Europe,national-research-tomsk-polytechnic-university
443,2018,441-450,Bangor University,NaN,United Kingdom,NaN,Europe,bangor-university
446,2018,441-450,Singapore Management University,NaN,Singapore,NaN,Asia,singapore-management-university
543,2018,501-550,University of Macau,NaN,Macau SAR,NaN,Asia,university-macau


In [12]:
# Count the NaN value in the city column
uni_rank_2022["city"].isnull().sum()

17

In [13]:
# Initialise a new list and copy over the data for the cleansing process
df = []
df = uni_rank_2022.copy()

In [14]:
# Split the city_extract column into multiple columns to get the city name
df["city_e1"] = df["city_extract"].str.split('-').str[0]
df["city_e2"] = df["city_extract"].str.split('-').str[1]
df["city_e3"] = df["city_extract"].str.split('-').str[2]
df["city_e4"] = df["city_extract"].str.split('-').str[3]
df["city_e5"] = df["city_extract"].str.split('-').str[4]
df["city_e6"] = df["city_extract"].str.split('-').str[5]
df[df["city"].isnull()]

,year,rank_display,university,score,country,city,region,city_extract,city_e1,city_e2,city_e3,city_e4,city_e5,city_e6
118,2018,119,Aarhus University,59.3,Denmark,NaN,Europe,aarhus-university,aarhus,university,NaN,NaN,NaN,NaN
201,2018,202,Queen's University Belfast,47.4,United Kingdom,NaN,Europe,queens-university-belfast,queens,university,belfast,NaN,NaN,NaN
241,2018,242,Université Paris-Saclay,42.3,France,NaN,Europe,universite-paris-saclay,universite,paris,saclay,NaN,NaN,NaN
256,2018,=256,Kyung Hee University,40.6,South Korea,NaN,Asia,kyung-hee-university,kyung,hee,university,NaN,NaN,NaN
346,2018,=346,Brunel University London,33.9,United Kingdom,NaN,Europe,brunel-university-london,brunel,university,london,NaN,NaN,NaN
360,2018,=361,Oxford Brookes University,33.1,United Kingdom,NaN,Europe,oxford-brookes-university,oxford,brookes,university,NaN,NaN,NaN
385,2018,=386,National Research Tomsk Polytechnic University,31.3,Russia,NaN,Europe,national-research-tomsk-polytechnic-university,national,research,tomsk,polytechnic,university,NaN
443,2018,441-450,Bangor University,NaN,United Kingdom,NaN,Europe,bangor-university,bangor,university,NaN,NaN,NaN,NaN
446,2018,441-450,Singapore Management University,NaN,Singapore,NaN,Asia,singapore-management-university,singapore,management,university,NaN,NaN,NaN
543,2018,501-550,University of Macau,NaN,Macau SAR,NaN,Asia,university-macau,university,macau,NaN,NaN,NaN,NaN


In [15]:
# Update the city column with the name extracted from the university URL
df.iloc[[118],[5]] = "Aarhus"
df.iloc[[201],[5]] = "Belfast"
df.iloc[[241],[5]] = "Paris"
df.iloc[[346],[5]] = "London"
df.iloc[[360],[5]] = "Brookes"
df.iloc[[385],[5]] = "Tomsk"
df.iloc[[443],[5]] = "Bangor"
df.iloc[[587],[5]] = "Paris"
df.iloc[[724],[5]] = "Olomouc"
df.iloc[[741],[5]] = "Brighton"
df.iloc[[916],[5]] = "Lille"
df.iloc[[933],[5]] = "Greenwich"
df.iloc[[970],[5]] = "Stockholm"

# Visit the university websites and look up the city names for country: South Korea, Singapore, Macau, China
df.iloc[[256],[5]] = "Suwon"
df.iloc[[446],[5]] = "Singapore"
df.iloc[[543],[5]] = "Macau"
df.iloc[[817],[5]] = "Changsha"

In [16]:
# Check whether there is any NaN in the city column
df.loc[(df["city"].isnull())] 

,year,rank_display,university,score,country,city,region,city_extract,city_e1,city_e2,city_e3,city_e4,city_e5,city_e6


In [17]:
# Drop the unused columns
df = df.drop(["city_e1","city_e2","city_e3","city_e4","city_e5","city_e6","city_extract"], axis=1)
df.head()

,year,rank_display,university,score,country,city,region
0,2018,1,Massachusetts Institute of Technology (MIT),100.0,United States,Cambridge,North America
1,2018,2,Stanford University,98.7,United States,Stanford,North America
2,2018,3,Harvard University,98.4,United States,Cambridge,North America
3,2018,4,California Institute of Technology (Caltech),97.7,United States,Pasadena,North America
4,2018,5,University of Cambridge,95.6,United Kingdom,Cambridge,Europe


## Transforming data (cleasing) - Column: Rank_Display & Score

In [18]:
# Split the unwanted characters, capture the ranking from the column rank_display and store the data in the new column
df["rank"] = df['rank_display'].str.split('-').str[0]
df["rank"] = df['rank_display'].str.split('+').str[0]
df.tail(10)

,year,rank_display,university,score,country,city,region,rank
967,2018,1001+,Karolinska Institutet,NaN,Sweden,Stockholm,Europe,1001
968,2018,1001+,King Abdullah University of Science & Technolo...,NaN,Saudi Arabia,Thuwal,Asia,1001
969,2018,1001+,London Business School,NaN,United Kingdom,London,Europe,1001
970,2018,1001+,Stockholm School of Economics,NaN,Sweden,Stockholm,Europe,1001
971,2018,1001+,Bocconi University,NaN,Italy,Milan,Europe,1001
972,2018,1001+,Université de Technologie de Compiègne (UTC),NaN,France,Compiègne,Europe,1001
973,2018,1001+,"University of California, San Francisco",NaN,United States,San Francisco,North America,1001
974,2018,1001+,Prague University of Economics and Business,NaN,Czech Republic,Prague,Europe,1001
975,2018,1001+,Weizmann Institute of Science,NaN,Israel,Rehovot,Asia,1001
976,2018,1001+,WHU - Otto Beisheim School of Management,NaN,Germany,Vallendar,Europe,1001


In [19]:
# Check whether the column rank contains any unwanted characters
df[df["rank"].str.match('^=.*')==True]

,year,rank_display,university,score,country,city,region,rank
20,2018,=21,Duke University,87.0,United States,Durham,North America,=21
21,2018,=21,University of Michigan-Ann Arbor,87.0,United States,Ann Arbor,North America,=21
22,2018,=23,King's College London,86.9,United Kingdom,London,Europe,=23
23,2018,=23,The University of Edinburgh,86.9,United Kingdom,Edinburgh,Europe,=23
27,2018,=28,Northwestern University,84.8,United States,Evanston,North America,=28
...,...,...,...,...,...,...,...,...
394,2018,=395,Illinois Institute of Technology,30.7,United States,Chicago,North America,=395
395,2018,=395,The American University in Cairo,30.7,Egypt,Cairo,Africa,=395
397,2018,=398,"Goldsmiths, University of London",30.5,United Kingdom,London,Europe,=398
398,2018,=398,Taipei Medical University (TMU),30.5,Taiwan,Taipei City,Asia,=398


In [20]:
# Remove all the unwanted characters
df["rank"].replace("=",'',regex=True, inplace=True)
df["rank"] = df['rank'].str.split('-').str[0]

# Check whether all the unwanted characters had been removed
df[df["rank"].str.match('^=.*')==True]

,year,rank_display,university,score,country,city,region,rank


In [21]:
# Fill all NaN values with 0 (zero)
df["score"] =  df["score"].fillna(0)
df["rank_display"] = df["rank_display"].fillna(0)
df["rank"] = df["rank"].fillna(0)

# Check to see whether there is any NaN value left in the dataframe
df.isnull().sum()

year            0
rank_display    0
university      0
score           0
country         0
city            0
region          0
rank            0
dtype: int64

In [22]:
# Convert the column rank to integer
df["rank"] = pd.to_numeric(df["rank"])

In [23]:
# Set index column as table "id" column
df.reset_index(drop=True,inplace=True)
df['id'] = df.index
df.set_index("id")

,year,rank_display,university,score,country,city,region,rank
id,,,,,,,,
0,2018,1,Massachusetts Institute of Technology (MIT),100.0,United States,Cambridge,North America,1
1,2018,2,Stanford University,98.7,United States,Stanford,North America,2
2,2018,3,Harvard University,98.4,United States,Cambridge,North America,3
3,2018,4,California Institute of Technology (Caltech),97.7,United States,Pasadena,North America,4
4,2018,5,University of Cambridge,95.6,United Kingdom,Cambridge,Europe,5
...,...,...,...,...,...,...,...,...
972,2018,1001+,Université de Technologie de Compiègne (UTC),0.0,France,Compiègne,Europe,1001
973,2018,1001+,"University of California, San Francisco",0.0,United States,San Francisco,North America,1001
974,2018,1001+,Prague University of Economics and Business,0.0,Czech Republic,Prague,Europe,1001


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          977 non-null    int64  
 1   rank_display  977 non-null    object 
 2   university    977 non-null    object 
 3   score         977 non-null    float64
 4   country       977 non-null    object 
 5   city          977 non-null    object 
 6   region        977 non-null    object 
 7   rank          977 non-null    int64  
 8   id            977 non-null    int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 68.8+ KB


## Transforming - Finalise DataFrames

In [25]:
uni_rank_transformed = []
uni_rank_transformed = df[["id","year","rank","score"]].copy()
uni_rank_transformed.set_index("id",drop=True, inplace=True)
uni_rank_transformed.head()

,year,rank,score
id,,,
0,2018,1,100.0
1,2018,2,98.7
2,2018,3,98.4
3,2018,4,97.7
4,2018,5,95.6


In [26]:
uni_details_transformed = []
uni_details_transformed = df[["id","university","city","country","region"]].copy()
uni_details_transformed.set_index("id",drop=True, inplace=True)
uni_details_transformed.head()

,university,city,country,region
id,,,,
0,Massachusetts Institute of Technology (MIT),Cambridge,United States,North America
1,Stanford University,Stanford,United States,North America
2,Harvard University,Cambridge,United States,North America
3,California Institute of Technology (Caltech),Pasadena,United States,North America
4,University of Cambridge,Cambridge,United Kingdom,Europe


## Create postgresql database connection

In [27]:
connection_string = "postgres:password@localhost:5432/QS_Uni_Ranking_db"
engine = create_engine(f'postgresql://{connection_string}')

In [28]:
# Confirm tables creation in pgAdmin4
engine.table_names()

['uni_rank', 'uni_details']

## Load pandas DataFrames into database

In [29]:
uni_rank_transformed.to_sql(name='uni_rank', con=engine, if_exists='append', index=True)

In [30]:
uni_details_transformed.to_sql(name='uni_details', con=engine, if_exists='append', index=True)